# Statistics generation and creation

Notebook to read simulation data, calculate statistics and aggregated statistics and save to separate csv files

In [5]:
from src.metrics import *
#from evaluation.src.data_loader import load_data
import pandas as pd
import numpy as numpy


In [6]:
results_pid_nominal = load_data('pid', 'nominal')
results_pid_with_noise = load_data('pid', 'noise')
results_pid_with_disturbances = load_data('pid', 'disturbances')

results_onoff_nominal = load_data('onoff', 'nominal')
results_onoff_with_noise = load_data('onoff', 'noise')
results_onoff_with_disturbances = load_data('onoff', 'disturbances')

results_fuzzy_nominal = load_data('fuzzy', 'nominal')
results_fuzzy_with_noise = load_data('fuzzy', 'noise')
results_fuzzy_with_disturbances = load_data('fuzzy', 'disturbances')

metrics = [steady_state_error, mean_square_error, overshoot, rise_time, settling_time, comfort_time, 
            energy_consumed,variance_after_settling, recovery_time, number_of_oscillations]

results = [results_pid_nominal, results_pid_with_noise, results_pid_with_disturbances,
           results_onoff_nominal, results_onoff_with_noise, results_onoff_with_disturbances,
           results_fuzzy_nominal, results_fuzzy_with_noise, results_fuzzy_with_disturbances]



NameError: name 'load_data' is not defined

In [ ]:
def calculate_and_export_stats(results_array, metrics, folder_path='statistics'):
    aggregated_stats = pd.DataFrame(columns=['metric', 'mean', 'mean_ci', 'var', 'var_ci'])
    stats_per_run = []
    for metric in metrics:
        metric_results = ([metric(results_array, 'measuredTemp', np.full(len(result), 20.0), ) for result in results_array])
        
        stats_per_run.append({
            'metric': metric.__name__,
            'values': metric_results
        })

        metric_entry ={
            'metric': metric.__name__,
            'mean': np.mean(metric_results),
            'mean_ci': np.std(metric_results) / np.sqrt(len(metric_results)),
            'var': np.var(metric_results),
            'var_ci': np.std(metric_results) / np.sqrt(len(metric_results))
        }
        aggregated_stats = pd.concat([aggregated_stats, pd.DataFrame([metric_entry])], ignore_index=True)

    aggregated_stats.to_csv(f"{folder_path}/{results_array}_aggregated.csv", index=False)
    stats_per_run_df = pd.DataFrame(stats_per_run)
    stats_per_run_df.to_csv(f"{folder_path}/{results_array}.csv", index=False)

In [ ]:
for result in results:
    calculate_and_export_stats(result, metrics)
    print(f"Stats for {result} calculated and exported.")
